In [1]:
import os
from os import listdir
from os.path import isfile, join
from pathlib import Path
import pandas as pd
import pickle
import re
import nltk
from nltk.tokenize import sent_tokenize
# nltk.download('all')

In [2]:
# Utility functions
def get_all_filenames(dir: str='./') -> list:
    ''' traverse root directory, and list directories as dirs and files as files. Return filenames in rootdir. '''
    files = [f for f in listdir(dir) if isfile(join(dir, f))]
    files.sort()
    return files

def flatten(t):
    return [item for sublist in t for item in sublist]

def lower(pharse: str) -> str:
    return pharse.lower()
    
reports = get_all_filenames()
reports = [r for r in reports if r.endswith('.txt')]
reports = [r for r in reports if r.startswith('Gafgyt')] # Darlloz
# reports.remove('Dofloo-360InternetSecurityCenter.txt')
print(reports)

['Gafgyt-fortinet.txt', 'Gafgyt-nozominetworks.txt', 'Gafgyt-sectrio.txt', 'Gafgyt-securityscorecard.txt', 'Gafgyt-trendmicro.txt', 'Gafgyt-unit42-2.txt', 'Gafgyt-unit42.txt', 'Gafgyt-wikipedia.txt']


將所有 report 斷句，另存csv檔  
- 有人為斷句的文章: `Dofloo-Trendmicro.txt`

In [6]:
Path("./sentence csvs").mkdir(parents=True, exist_ok=True)

for rname in reports:
    with open(rname, 'r', encoding="utf-8") as f:
        raw_text = f.read()
    # raw_text = lower(raw_text)
    text = re.sub(r"\"", '', raw_text)
    text_sentences = sent_tokenize(text, language='english')

    must_split_ids = [] # ~~ 符號表示人為強制斷句，若文本中有 ~~ 則表示nltk斷得不夠徹底需再進行強制斷句
    if text.find('~~') != -1:
        tmp = text_sentences
        text_sentences = []
        for i, sent in enumerate(tmp): 
            try:
                text_sentences.extend(sent.split('~~'))
            except:
                text_sentences.append(sent)
        # print(text_sentences)

    # remove empty sentence
    text_sentences = [s for s in text_sentences if len(s) != 0]
    
    [sent.strip() for sent in text_sentences] # build up csv file
    lst = []
    for i, sent in enumerate(text_sentences):
        lst.append([f'Sentence {i+1}', sent])
    rname_wo_txt = rname.split('.')[0] # 去掉 .txt
    df = pd.DataFrame(lst, columns=['Number','Content'])
    df.to_csv(f'./sentence csvs/{rname_wo_txt}.csv', index=False)

測試將一篇 report  斷句

In [4]:
with open(reports[2], 'r', encoding="utf-8") as f:
    raw_text = f.read()
# raw_text = '''"When you start coding, it makes you feel smart in itself, like you're in the Matrix [film]," says Janine Luk, a 26 year-old software engineer who works in London. Born in Hong Kong, she started her career in yacht marketing in the south of France but found is's "a bit repetitive and superficial". So, she started teaching herself to code after work, followed by a 15-week coding boot camp.'''

# removing double quotes
text = re.sub(r"\"", '', raw_text)

# breaking text into sentences
text_sentences = sent_tokenize(text, language='english')

# printing out sentences
for i, sent in enumerate(text_sentences):
    print("Sentence {}: {}".format(i + 1, sent), end = "\n\n")

Sentence 1: Overview

Gafgyt malware, also known as Bashlite, is a Trojan horse that attacks *nix systems, targeting IoT devices such as routers and open backdoors into compromised systems, steals information, and enlists infected devices into botnets that provide Distributed Denial of Service (DDoS).

Sentence 2: The Gafgyt malware was first introduced in 2014, when it exploited unknown vulnerabilities in small home and small office routers to launch Distributed Denial of Service (DDoS) attacks, similar to the well-know Mirai botnet.

Sentence 3: There have been numerous variants of Gafgyt that have appeared since 2014, also referred to by the names of BASHLITE, Lizkebab, Torlus, and Qbot).

Sentence 4: This new variant of Gafgyt malware includes Mirai malware modules and added new modules to launch DDoS attacks against victims.

Sentence 5: ~~
Infection Flow

Gafgyt malware infect most of systems or IoT devices through the bash script which can download the main payload of the malwar